<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Изучение-общей-информации-по-данным" data-toc-modified-id="Изучение-общей-информации-по-данным-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение общей информации по данным</a></span></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обработка данных</a></span></li><li><span><a href="#Деление-выборки" data-toc-modified-id="Деление-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Деление выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-решающего-дерева" data-toc-modified-id="Модель-решающего-дерева-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель решающего дерева</a></span></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score 
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# загрузим данные
df = pd.read_csv('/datasets/toxic_comments.csv')


### Изучение общей информации по данным

In [3]:
# размер данных
df.shape

(159292, 3)

In [4]:
# общая информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df.tail()

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [7]:
df.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [8]:
# проверим пропуски, их нет
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [9]:
df.duplicated().sum() #посмотрим наличие дубликатов, не наблюдаются

0

In [10]:
# посмотрим на количество строк с разными классами в столбце целевого признака, выявлено соотношение 90/10
df['toxic'].value_counts(normalize=True) 

0    0.898388
1    0.101612
Name: toxic, dtype: float64

### Обработка данных

In [11]:
# Перевод в нижний регистр
df['text_lower'] = df['text'].str.lower()

In [12]:
%%time
# Удаление ненужных символов
translator = str.maketrans('', '', string.punctuation + string.digits)
df['text_no_punct'] = df['text_lower'].apply(lambda x: x.translate(translator))

CPU times: user 1.49 s, sys: 44 ms, total: 1.53 s
Wall time: 1.54 s


In [13]:
%%time
# Удаление стоп-слов
stop_words=set(stopwords.words('english'))
stop_words.add('n')
df['text_no_stopwords'] = df['text_no_punct'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

CPU times: user 51.7 s, sys: 157 ms, total: 51.8 s
Wall time: 52 s


In [14]:
# первый вариант

# функция определения тега слов 
def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# функция лемматизации 
def lemma_function(text):
    m = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemma = ' '.join([m.lemmatize(word, get_pos(word)) for word in word_list])
    return lemma

In [15]:
df['lemm_text'] = df['text_no_stopwords'].progress_apply(lemma_function)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [16]:
# второй вариант

# функция определения тега слов 
def pos(tag):
    postag = {'NN':'n', 'JJ':'a',
              'VB':'v', 'RB':'r'}
    try:
        return postag[tag[:2]]
    except:
        return 'n' 
# функция лемматизации 
def lemmatize_sent(text): 
    m = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemma = ' '.join([m.lemmatize(word, pos=pos(tag))
            for word, tag in pos_tag(word_list)])
    return lemma


In [17]:
df['lemm2_text'] = df['text_no_stopwords'].progress_apply(lemmatize_sent)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [18]:
df.head()

,Unnamed: 0,text,toxic,text_lower,text_no_punct,text_no_stopwords,lemm_text,lemm2_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation\nwhy the edits made under my usern...,explanation\nwhy the edits made under my usern...,explanation edits made username hardcore metal...,explanation edits make username hardcore metal...,explanation edits make username hardcore metal...
1,1,D'aww! He matches this background colour I'm s...,0,d'aww! he matches this background colour i'm s...,daww he matches this background colour im seem...,daww matches background colour im seemingly st...,daww match background colour im seemingly stuc...,daww match background colour im seemingly stuc...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"hey man, i'm really not trying to edit war. it...",hey man im really not trying to edit war its j...,hey man im really trying edit war guy constant...,hey man im really try edit war guy constantly ...,hey man im really try edit war guy constantly ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,"""\nmore\ni can't make any real suggestions on ...",\nmore\ni cant make any real suggestions on im...,cant make real suggestions improvement wondere...,cant make real suggestion improvement wonder s...,cant make real suggestion improvement wonder s...
4,4,"You, sir, are my hero. Any chance you remember...",0,"you, sir, are my hero. any chance you remember...",you sir are my hero any chance you remember wh...,sir hero chance remember page thats,sir hero chance remember page thats,sir hero chance remember page thats


In [19]:
df['lemm_text']

0         explanation edits make username hardcore metal...
1         daww match background colour im seemingly stuc...
2         hey man im really try edit war guy constantly ...
3         cant make real suggestion improvement wonder s...
4                       sir hero chance remember page thats
                                ...                        
159287    second time ask view completely contradicts co...
159288                 ashamed horrible thing put talk page
159289    spitzer umm there actual article prostitution ...
159290    look like actually put speedy first version de...
159291    really dont think understand come idea bad rig...
Name: lemm_text, Length: 159292, dtype: object

### Деление выборки

In [20]:
# выделим признаки
features = df['lemm_text']

In [21]:
# выделим целевой признак
target = df['toxic']

In [22]:
# посмотрим на количество строк с разными классами в столбце целевого признака, выявлено соотношение 90/10
target.value_counts(normalize=True) 

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [23]:
# разделим данные на 90/10
features_train, features_test, target_train, target_test = train_test_split(features, target, \
                                                                                test_size = 0.1, random_state=12345, \
                                                                                stratify=target)

In [24]:
# проверим данные
print(f"Количество строк в target_train по классам: {np.bincount(target_train)}")
print(f"Количество строк в target_test по классам: {np.bincount(target_test)}")

Количество строк в target_train по классам: [128795  14567]
Количество строк в target_test по классам: [14311  1619]


In [25]:
# проверим разбиение
features_train.shape, features_test.shape

((143362,), (15930,))

**Вывод по разделу**

Ознакомились с данными
- строк 159292, столбца 3
- пропусков нет, дубликатов нет
- установлен дисбаланс классов 90/10 в целевом признаке

Провели обработку данных
- перевод в нижний регистр
- удаление лишних символов
- удаление стоп-слов
- лемматизация текста

Разделили выборки
- выделили признаки и целевой признак
- выявили дисбаланс классов в целевом признаке, выявлено соотношение 90/10
- разделили выборки на 90/10, учли дисбаланс классов

## Обучение

### Модель логистической регрессии

In [26]:
f1_tfidf = []
f1_bow = []

In [27]:
%%time
# Обучим модель логистической регрессии с векторизацией признаков с TF-IDF
classifier = LogisticRegression(random_state=12345, max_iter=1000)
vectorizer = TfidfVectorizer()
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])
parameters = {
    'classifier__C': [1, 10],
}
grid_lr_tf = GridSearchCV(pipeline, parameters, cv=3, scoring='f1', n_jobs=-1, verbose=10)
grid_lr_tf.fit(features_train, target_train)
best_params = grid_lr_tf.best_params_
best_score = grid_lr_tf.best_score_
print(f'Лучшие гиперпараметры с TF-IDF:  {best_params}')
print(f'F1-score на тренировочной с TF-IDF:  {best_score}')
f1_tfidf.append(round(best_score, 3))


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3; 1/2] START classifier__C=1.............................................
[CV 1/3; 1/2] END ...........................classifier__C=1; total time=  53.9s
[CV 2/3; 1/2] START classifier__C=1.............................................
[CV 2/3; 1/2] END ...........................classifier__C=1; total time=  41.6s
[CV 3/3; 1/2] START classifier__C=1.............................................
[CV 3/3; 1/2] END ...........................classifier__C=1; total time= 1.0min
[CV 1/3; 2/2] START classifier__C=10............................................
[CV 1/3; 2/2] END ..........................classifier__C=10; total time= 1.9min
[CV 2/3; 2/2] START classifier__C=10............................................
[CV 2/3; 2/2] END ..........................classifier__C=10; total time= 1.6min
[CV 3/3; 2/2] START classifier__C=10............................................
[CV 3/3; 2/2] END ..........................classi

In [28]:
%%time
# Обучим модель логистической регрессии с векторизацией признаков BOW Создание мешка слов
classifier = LogisticRegression(random_state=12345, max_iter=1000)
vectorizer = CountVectorizer()
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])
parameters = {
    'classifier__C': [1, 10],
}
grid_lr_bow = GridSearchCV(pipeline, parameters, cv=3, scoring='f1', n_jobs=-1, verbose=10)
grid_lr_bow.fit(features_train, target_train)
best_params = grid_lr_bow.best_params_
best_score = grid_lr_bow.best_score_
print(f'Лучшие гиперпараметры с BOW:  {best_params}')
print(f'F1-score на тренировочной с BOW:  {best_score}')
f1_bow.append(round(best_score, 3))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3; 1/2] START classifier__C=1.............................................
[CV 1/3; 1/2] END ...........................classifier__C=1; total time= 3.8min
[CV 2/3; 1/2] START classifier__C=1.............................................
[CV 2/3; 1/2] END ...........................classifier__C=1; total time= 3.6min
[CV 3/3; 1/2] START classifier__C=1.............................................
[CV 3/3; 1/2] END ...........................classifier__C=1; total time= 3.8min
[CV 1/3; 2/2] START classifier__C=10............................................


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/3; 2/2] END ..........................classifier__C=10; total time= 8.3min
[CV 2/3; 2/2] START classifier__C=10............................................


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3; 2/2] END ..........................classifier__C=10; total time= 8.4min
[CV 3/3; 2/2] START classifier__C=10............................................


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3; 2/2] END ..........................classifier__C=10; total time= 8.3min
Лучшие гиперпараметры с BOW:  {'classifier__C': 10}
F1-score на тренировочной с BOW:  0.7606236385345668
CPU times: user 16min 13s, sys: 29min 54s, total: 46min 7s
Wall time: 46min 11s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


  ### Модель решающего дерева

In [29]:
%%time
# Обучим модель решающего дерева с векторизацией признаков с TF-IDF
classifier = DecisionTreeClassifier(random_state=12345)
vectorizer = TfidfVectorizer()
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])
parameters = {
    'classifier__max_depth': [5, 10],  
}
grid_tree_tf = GridSearchCV(pipeline, parameters, cv=3, scoring='f1', n_jobs=-1, verbose=10)
grid_tree_tf.fit(features_train, target_train)
best_params = grid_tree_tf.best_params_
best_score = grid_tree_tf.best_score_
print(f'Лучшие гиперпараметры с TF-IDF:  {best_params}')
print(f'F1-score на тренировочной с TF-IDF:  {round(best_score, 3)}')
f1_tfidf.append(round(best_score, 3))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3; 1/2] START classifier__max_depth=5.....................................
[CV 1/3; 1/2] END ...................classifier__max_depth=5; total time=   7.8s
[CV 2/3; 1/2] START classifier__max_depth=5.....................................
[CV 2/3; 1/2] END ...................classifier__max_depth=5; total time=   7.7s
[CV 3/3; 1/2] START classifier__max_depth=5.....................................
[CV 3/3; 1/2] END ...................classifier__max_depth=5; total time=   7.9s
[CV 1/3; 2/2] START classifier__max_depth=10....................................
[CV 1/3; 2/2] END ..................classifier__max_depth=10; total time=  10.7s
[CV 2/3; 2/2] START classifier__max_depth=10....................................
[CV 2/3; 2/2] END ..................classifier__max_depth=10; total time=  10.8s
[CV 3/3; 2/2] START classifier__max_depth=10....................................
[CV 3/3; 2/2] END ..................classifier__ma

In [30]:
%%time
# Обучим модель решающего дерева с векторизацией признаков BOW Создание мешка слов
classifier = DecisionTreeClassifier(random_state=12345)
vectorizer = CountVectorizer()
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])
parameters = {
    'classifier__max_depth': [5, 10],  
}
grid_tree_bow = GridSearchCV(pipeline, parameters, cv=3, scoring='f1', n_jobs=-1, verbose=10)
grid_tree_bow.fit(features_train, target_train)
best_params = grid_tree_bow.best_params_
best_score = grid_tree_bow.best_score_
print(f'Лучшие гиперпараметры с BOW:  {best_params}')
print(f'F1-score на тренировочной с BOW:  {best_score}')
f1_bow.append(round(best_score, 3))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3; 1/2] START classifier__max_depth=5.....................................
[CV 1/3; 1/2] END ...................classifier__max_depth=5; total time=   6.5s
[CV 2/3; 1/2] START classifier__max_depth=5.....................................
[CV 2/3; 1/2] END ...................classifier__max_depth=5; total time=   6.5s
[CV 3/3; 1/2] START classifier__max_depth=5.....................................
[CV 3/3; 1/2] END ...................classifier__max_depth=5; total time=   6.5s
[CV 1/3; 2/2] START classifier__max_depth=10....................................
[CV 1/3; 2/2] END ..................classifier__max_depth=10; total time=   8.2s
[CV 2/3; 2/2] START classifier__max_depth=10....................................
[CV 2/3; 2/2] END ..................classifier__max_depth=10; total time=   8.7s
[CV 3/3; 2/2] START classifier__max_depth=10....................................
[CV 3/3; 2/2] END ..................classifier__ma

In [31]:
df_total = pd.DataFrame({'Модель':['Линейная регрессия', 'Модель Дерево решений'], \
                         'F1-score на тренировочной с BOW:':f1_bow,
                         'F1-score на тренировочной с TF-IDF:':f1_tfidf})
df_total

,Модель,F1-score на тренировочной с BOW:,F1-score на тренировочной с TF-IDF:
0,Линейная регрессия,0.761,0.774
1,Модель Дерево решений,0.613,0.610


**Вывод по разделу**

Обучили две модели с векторизацией Мешка слов и TF-IDF и подбора параметров GridSearchCV, выбрали лучшие параметры и оценили качество моделей.

Наилучший результат показала модель Логистическая регрессия с параметрами (random_state=12345, max_iter=1000, C=10) с векторизацией с TF-IDF

F1-score на тренировочной 0.774

### Проверка модели на тестовой выборке

In [33]:
prediction = best_pipeline.predict(features_test)
f11 = f1_score(target_test, prediction)
print('Тестовая выборка')
print('F1-score на тесте:', round(f11, 3))

Тестовая выборка
F1-score на тесте: 0.791


## Выводы

Проанализировав данные Интернет-магазин «Викишоп» удалось построить модель, которая будет искать токсичные комментарии и отправлять их на модерацию 

Модель Логистическая регрессия с параметрами (random_state=12345, max_iter=1000, C=10) с векторизацией с TF-IDF
со значением F1 = 0.791

в 1 разделе ознакомились с данными, выяснили, что:
- строк 159292, столбца 3
- пропусков нет, дубликатов нет
- установлен дисбаланс классов 90/10 в целевом признаке

Провели обработку данных
- перевод в нижний регистр
- удаление лишних символов
- удаление стоп-слов
- лемматизация текста

Разделили выборки
- выделили признаки и целевой признак
- выявили дисбаланс классов в целевом признаке, выявлено соотношение 90/10
- разделили выборки на 90/10, учли дисбаланс классов


во 2 разделе обучили две модели с векторизацией Мешка слов и TF-IDF и подбора параметров GridSearchCV, выбрали лучшие параметры и оценили качество моделей.

Наилучший результат показала модель Логистическая регрессия с параметрами (random_state=12345, max_iter=1000, C=10) с векторизацией с TF-IDF

F1-score на тренировочной 0.774

Протестировали лучшую модель на тестовой выборке:

F1-score на тестовой выборке 0.791
